In [207]:

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [208]:
df_train = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_data.csv")

In [209]:
df_train.describe()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Cabin,Embarked,Age
count,1493.000000,1493.000000,1493.000000,1493.000000,1493.000000,1493.000000,1493.000000,1493.000000,1.488000e+03
mean,597.697254,0.327528,2.261219,0.690556,0.184863,0.178835,5.994642,1.834561,2.151174e+05
std,251.365652,0.469469,0.628908,0.462419,0.594308,0.592203,24.214966,0.538178,8.295612e+06
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-3.200000e+04
25%,377.000000,0.000000,2.000000,0.000000,0.000000,0.000000,-1.000000,2.000000,-3.500000e+01
50%,758.000000,0.000000,2.000000,1.000000,0.000000,0.000000,-1.000000,2.000000,1.600000e+01
75%,813.000000,1.000000,3.000000,1.000000,0.000000,0.000000,-1.000000,2.000000,2.400000e+01
max,891.000000,1.000000,3.000000,1.000000,5.000000,6.000000,133.000000,2.000000,3.200000e+08


In [210]:
(df_train["Age"] < 0).sum()

679

In [211]:
df_train.drop(df_train[df_train.Age < 0].index, inplace = True)

In [212]:
(df_train["Age"] < 0).sum()

0

In [213]:
ageNa = df_test["Age"].isna().sum()

print(f"Ammount of Nan Values in Age Series: {ageNa}")

Ammount of Nan Values in Age Series: 0


In [214]:
(df_train["Age"] > 100).sum()

6

In [215]:
df_train.drop(df_train[df_train.Age > 100].index, inplace = True)

In [216]:
df_train["PassengerId"] = df_train["PassengerId"].drop_duplicates()

In [217]:
def sjekk_df(x):
    try:
        return float(x)
    except:
        return -1 

In [218]:

df_train["Fare"] = df_train.Fare.apply(lambda x: sjekk_df(x))

In [219]:
df_train["Ticket"] = df_train["Ticket"].drop_duplicates()

In [220]:
df_train = df_train.dropna()

In [221]:
df_train["Ticket"] = df_train.Fare.apply(lambda x: sjekk_df(x))

In [222]:
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 436 entries, 0 to 1492
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  436 non-null    float64
 1   Survived     436 non-null    int64  
 2   Pclass       436 non-null    int64  
 3   Sex          436 non-null    int64  
 4   SibSp        436 non-null    int64  
 5   Parch        436 non-null    int64  
 6   Ticket       436 non-null    float64
 7   Fare         436 non-null    float64
 8   Cabin        436 non-null    int64  
 9   Embarked     436 non-null    int64  
 10  Age          436 non-null    float64
dtypes: float64(4), int64(7)
memory usage: 40.9 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  143 non-null    int64  
 1   Survived     143 non-null    int64  
 2   Pclass       143 no

In [223]:
model = xgb.XGBRegressor()

In [224]:
X_train = df_train.drop("Age", axis=1)
y_train = df_train["Age"]

In [225]:
X_train, X_train_test, y_train, y_train_Test = train_test_split(X_train, y_train, train_size=0.8, random_state=42)

In [226]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [227]:
preds = model.predict(X_train_test)

In [228]:
mse = mean_squared_error(preds, y_train_Test)
mse_root = np.sqrt(mse)

In [229]:
print(mse_root)

14.351054823483793


In [230]:
params = {
    "learning_rate": [0.300000012],
    "max_depth": [256],
    "min_child_weight": [1],
    "gamma": [0.0],
    "colsample_bytree": [1],
    "n_estimators": [100],
}

In [231]:
model2 = xgb.XGBRegressor()

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=250, scoring="neg_mean_squared_error", n_jobs=-1, cv=5)

# Fit the model with x and y train sets
random_search.fit(X_train, y_train)

C:\Users\Jakso\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 1 is smaller than n_iter=250. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          callbacks=None, colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0, gpu_id=-1,
                                          grow_policy='depthwise',
                                          importance_type=None,
                                          interaction_constraints='',
                                          learning_rate=0.3000...
                                          max_depth=6, max_leaves=0,
                                          min_child_weight=1, missing=nan,
                                          monotone_constraints='()',
                                          n_estimators=100, n_jobs=0,
                                          num_parallel_tree=1, predictor='auto',
                                          random_state=0, ...),
                   n_iter=250, n_jobs=-1,
                   param_distributions={'colsample_bytree': [1], 'gamma': [0.0],
                                        'learning_rate': [0.300000012],
                                        'max_depth': [256],
                                        'min_child_weight': [1],
                                        'n_estimators': [100]},
                   scoring='neg_mean_squared_error')

In [232]:
model_new = random_search.best_params_
model_new

{'n_estimators': 100,
 'min_child_weight': 1,
 'max_depth': 256,
 'learning_rate': 0.300000012,
 'gamma': 0.0,
 'colsample_bytree': 1}

In [233]:
model_new = random_search.best_estimator_

In [234]:
preds = model_new.predict(X_train_test)

In [235]:
mse_new = mean_squared_error(preds, y_train_Test)
mse_root_new = np.sqrt(mse_new)
print(mse_root_new)

14.637636581217231


In [240]:
X_test = df_test.drop("Age", axis=1)
Y_test = df_test["Age"]

In [249]:

preds_test = model_new.predict(X_test)
mse_test=mean_squared_error(preds,Y_test)
mse_test_root = np.sqrt(mse_test)
print(mse_test_root)


ValueError: Found input variables with inconsistent numbers of samples: [88, 143]